In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import openai

from keymaker.models import chatgpt, gpt4, LlamaCpp
from keymaker.constraints import RegexConstraint, OptionsConstraint, StopsConstraint
from keymaker import Prompt, Completion, CompletionConfig

import json

with open("./examples/config.json") as f:
    openai.api_key = json.loads(f.read())["OPENAI_API_KEY"]

<IPython.core.display.Javascript object>

In [3]:
from typing import Optional


async def print_stream(completion: Optional[Completion]):
    if completion:
        print(repr(completion))


async def yo_stream(completion: Optional[Completion]):
    if completion:
        print("YO " + completion)

<IPython.core.display.Javascript object>

In [4]:
chat_model = chatgpt()

llama_model = LlamaCpp(
    model_path="/Users/nick/Downloads/llama-2-7b-chat.ggmlv3.q3_K_S.bin"
)

llama.cpp: loading model from /Users/nick/Downloads/llama-2-7b-chat.ggmlv3.q3_K_S.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 11 (mostly Q3_K - Small)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 4603.09 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  =  256.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 


<IPython.core.display.Javascript object>

In [5]:
from datetime import datetime

user_message = "Hi, my name is Nick."

<IPython.core.display.Javascript object>

In [6]:
my_math_answer = None


def store_my_math(answer):
    global my_math_answer
    my_math_answer = int(answer)
    if my_math_answer != 15:
        return "Duh me no know."
    return 15

<IPython.core.display.Javascript object>

In [7]:
# default model, stream and max_tokens for all completions
prompt = Prompt(
    """Time: {time}
User: {user_msg}
Assistant: Hello, {}{punctuation}
User: Can you write me a poem about a superhero named pandaman being a friend to {}?
Assistant:{poem}
User: What is 10+5?
Assistant: The answer is 10+5={math}

The final answer is {fin}!
""",
    chat_model,
    stream=print_stream,
    max_tokens=25,
)

filled_in = await prompt.format(
    # custom constraint
    CompletionConfig(constraint=OptionsConstraint({"Sam", "Nick"}), stream=yo_stream),
    lambda p: p.completions[0],
    # can do formatting just like a regular python string
    punctuation="!",
    user_msg=user_message,
    time=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    poem=CompletionConfig(
        llama_model,
        max_tokens=250,
        constraint=StopsConstraint("User|Assistant", include=False),
    ),
    math=CompletionConfig(
        llama_model,
        constraint=RegexConstraint("[0-9]+", terminate_on_match=False),
        map_fn=store_my_math,
    ),
    fin=lambda p: CompletionConfig(
        llama_model,
        constraint=RegexConstraint(rf"{p.completions.math}|16"),
        map_fn=store_my_math,
    ),
)

YO Nick



llama_print_timings:        load time =  9736.54 ms
llama_print_timings:      sample time =     0.69 ms /     1 runs   (    0.69 ms per token,  1459.85 tokens per second)
llama_print_timings: prompt eval time =  9736.51 ms /    69 tokens (  141.11 ms per token,     7.09 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  9822.76 ms
Llama.generate: prefix-match hit


Completion(text=' Of', start=6, stop=9, name=poem, chunk=True, score=0.9953074114416708)



llama_print_timings:        load time =  9736.54 ms
llama_print_timings:      sample time =     0.71 ms /     1 runs   (    0.71 ms per token,  1408.45 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  1834.80 ms /     1 runs   ( 1834.80 ms per token,     0.55 tokens per second)
llama_print_timings:       total time =  1837.49 ms
Llama.generate: prefix-match hit


Completion(text=' course', start=17, stop=24, name=poem, chunk=True, score=0.9998058923972919)


...

Completion(text='5', start=24, stop=25, name=math, chunk=True, score=0.9391089756408056)
Completion(text='1', start=25, stop=26, name=math, chunk=True, score=0.9987798325807007)



llama_print_timings:        load time =  9736.54 ms
llama_print_timings:      sample time =     0.70 ms /     1 runs   (    0.70 ms per token,  1426.53 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   101.81 ms /     1 runs   (  101.81 ms per token,     9.82 tokens per second)
llama_print_timings:       total time =   105.90 ms
Llama.generate: prefix-match hit


Completion(text='1', start=26, stop=27, name=math, chunk=True, score=0.9627398958975784)
Completion(text='1', start=2, stop=3, name=fin, chunk=True, score=0.9999998834967918)
Completion(text='6', start=3, stop=4, name=fin, chunk=True, score=0.9999999394308399)



llama_print_timings:        load time =  9736.54 ms
llama_print_timings:      sample time =     0.71 ms /     1 runs   (    0.71 ms per token,  1404.49 tokens per second)
llama_print_timings: prompt eval time =   986.05 ms /    13 tokens (   75.85 ms per token,    13.18 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  1004.86 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  9736.54 ms
llama_print_timings:      sample time =     0.71 ms /     1 runs   (    0.71 ms per token,  1414.43 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   138.64 ms /     1 runs   (  138.64 ms per token,     7.21 tokens per second)
llama_print_timings:       total time =   142.81 ms


<IPython.core.display.Javascript object>

In [8]:
filled_in

Prompt('Time: 2023-07-21 23:02:54
User: Hi, my name is Nick.
Assistant: Hello, Nick!
User: Can you write me a poem about a superhero named pandaman being a friend to Nick?
Assistant: Of course, I'd be happy to help! Here's a poem for you:
Pandaman, the hero of the land,
A friend to Nick, his heart so grand.
With powers beyond compare,
He fights for justice, without fear.

Nick and Pandaman, a bond so strong,
Together they fight, where right is wrong.
Their friendship grows with each passing day,
As they stand together, come what may.

Pandaman, the hero of the night,
A friend to Nick, shining so bright.
With powers beyond compare,
He fights for justice, without fear.

User
User: What is 10+5?
Assistant: The answer is 10+5=Duh me no know.

The final answer is Duh me no know.!
')

<IPython.core.display.Javascript object>

In [9]:
filled_in.completions.poem

Completion(text=' Of course, I'd be happy to help! Here's a poem for you:
Pandaman, the hero of the land,
A friend to Nick, his heart so grand.
With powers beyond compare,
He fights for justice, without fear.

Nick and Pandaman, a bond so strong,
Together they fight, where right is wrong.
Their friendship grows with each passing day,
As they stand together, come what may.

Pandaman, the hero of the night,
A friend to Nick, shining so bright.
With powers beyond compare,
He fights for justice, without fear.

User', start=174, stop=673, name=poem, chunk=False, score=7.346943203117714e-15)

<IPython.core.display.Javascript object>

In [10]:
filled_in.completions.math

'Duh me no know.'

<IPython.core.display.Javascript object>